<a href="https://colab.research.google.com/github/SahilTuladhar/TensorFlow-and-Pytorch--Practice/blob/master/01_PyTorch_Workflow_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/SahilTuladhar/TensorFlow-and-Pytorch--Practice.git

Cloning into 'TensorFlow-and-Pytorch--Practice'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 54 (delta 9), reused 38 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 1023.12 KiB | 3.16 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
!pwd

/content


In [4]:
cd TensorFlow-and-Pytorch--Practice

/content/TensorFlow-and-Pytorch--Practice


In [6]:
cd pytorch

/content/TensorFlow-and-Pytorch--Practice/pytorch


In [9]:
!git add '01.PyTorch Workflow Fundamentals.ipynb'

fatal: pathspec '01.PyTorch Workflow Fundamentals.ipynb' did not match any files


In [10]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
